### Библиотеки

In [7]:
import requests
from bs4 import BeautifulSoup
import csv
from time import sleep
from random import randint

import pandas as pd

### Параметры

In [8]:
# BASE_URL = "https://www.chitai-gorod.ru/search?phrase=психология&page="
BASE_URL = 'https://www.chitai-gorod.ru/catalog/books/klassicheskaya-proza-110003?page='
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}
PAGES = 10
OUTPUT_FILE = "books.csv"

### Функции для парсинга фичей на странице

#### Новая и старая цена

In [9]:
def parse_price(container):
    prices = []
    for part in container.stripped_strings:
        if '₽' in part:
            prices.append(part.replace('₽', '').replace('\xa0', '').strip())
    return {
        'current': prices[0] if len(prices) > 0 else None,
        'old': prices[1] if len(prices) > 1 else None
    }

#### Название книги

In [10]:
def parse_title(container):
    title_tag = container.find('h1')
    title = title_tag.get_text(strip=True)
    
    return title

#### Оценки

In [11]:
def parse_reviews(soup):

    # средняя оценка
    try:
        avg_rating = soup.find('span', class_="product-review-range__count").text.strip()
    except:
        avg_rating = None

    # количество оценок
    try:
        cnt_reviews = soup.find('div', class_='star-rating__statistics').text.replace('\xa0', '').strip()
    except:
        cnt_reviews = None
    
    return {
        'avg_rating': avg_rating,
        'cnt_reviews': cnt_reviews
    }


#### Характеристики книги

In [12]:
def parse_article(soup):
    divs = []
    result = {}

    for div in soup.select('section div', class_='product-detail-features__item'):
        divs.append(div)
    
    for div in divs:
        # # Обработка описания
        # article = div.find('article')
        # if article:
        #     description = ' '.join(article.stripped_strings)
        #     result['Описание'] = description
        #     continue
        
        # Обработка информационных блоков
        spans = div.find_all('span', limit=1)
        if not spans:
            continue
            
        key = spans[0].get_text(strip=True)
        value_element = spans[0].find_next_sibling()
        
        if not value_element:
            continue
            
        value = ' '.join(value_element.stripped_strings)
        result[key] = value
    
    return result

### Парсер одной книги

In [13]:
def parse_product_page(url, HEADERS=HEADERS):
    response = requests.get(url, headers=HEADERS)
    if response.status_code != 200:
        return {}
    
    soup = BeautifulSoup(response.text, "html.parser")
    result = {}

    # Ссылка
    result['link'] = url
    
    # Цена
    try:
        price_container = soup.find("div", class_="product-offer-price")
        price_data = parse_price(price_container)
        current_price = f"{price_data['current']} ₽" if price_data['current'] else None
        old_price = f"{price_data['old']} ₽" if price_data['old'] else None
    except:
        current_price = None
        old_price = None

    result['current_price'] = current_price
    result['old_price'] = old_price


    # Оценки
    try:
        review_data = parse_reviews(soup)
    except:
        review_data = {}

    result.update(review_data)
        

    # Название
    try:
        title_container = soup.find("div", class_="detail-product__header")
        title = parse_title(title_container)
        # title_tag = title_container.find('h1')
        # title = title_tag.get_text(strip=True)
    except:
        title = None

    result['title'] = title


    # Автор
    try:
        author_container = soup.find('div', class_="product-info-authors detail-product__header-authors")
        author = author_container.text.strip().split(',')[0]
    except:
        author = None
    
    result['author'] = author


    # Характеристики
    try:
        article_data = parse_article(soup)
    except:
        article_data = {}
    
    result.update(article_data)
    
    return result

### Парсер страницы

In [14]:
from tqdm import tqdm

def parse_page(BASE_URL, HEADERS, page_num):
    books = []

    # страница с выдачей
    url = f"{BASE_URL}{page_num}"
    response = requests.get(url, headers=HEADERS)

    # открываем страницу
    if response.status_code != 200:
        print(f"Ошибка {response.status_code} на странице {page_num}")
        return []
    
    # суп страницы
    soup = BeautifulSoup(response.text, "html.parser")
    
    # проходимся по книгам
    for item in tqdm(soup.find_all("article", class_="product-card")):

        # ищем ссылку на страницу книги
        links = []
        try:  
            # Находим тег <a> с атрибутом href, содержащим "/product/"
            link_tag = item.find("a", href=lambda x: x and x.startswith("/product/"))
            link = "https://www.chitai-gorod.ru" + link_tag["href"]
        except Exception as e:
            link = "Нет ссылки"
        
        # парсим страницу книги
        book_data = parse_product_page(link)
        
        # добавляем инфу по книге в список
        books.append(book_data)
        
    return books

### Функция большого парсера

In [17]:
def parse_site(BASE_URL=BASE_URL, HEADERS=HEADERS, PAGES=PAGES):
    all_books = []

    for page_num in range(1, PAGES + 1):
        print(f'Парсим страницу номер: {page_num}')
        
        books_page = parse_page(BASE_URL=BASE_URL, HEADERS=HEADERS, page_num=page_num)
    
        all_books += books_page
        print(len(all_books))

    return pd.DataFrame(all_books)

In [16]:
def save_data(all_books, OUTPUT_FILE):
    all_books.to_csv(OUTPUT_FILE, index=False)

In [18]:
result = parse_site(PAGES=60)
result

Парсим страницу номер: 1


100%|██████████| 60/60 [00:43<00:00,  1.37it/s]


60
Парсим страницу номер: 2


100%|██████████| 60/60 [00:42<00:00,  1.40it/s]


120
Парсим страницу номер: 3


...

100%|██████████| 60/60 [00:41<00:00,  1.46it/s]


3540
Парсим страницу номер: 60


100%|██████████| 60/60 [00:41<00:00,  1.43it/s]

3600


,link,current_price,old_price,avg_rating,cnt_reviews,title,author,ID товара,Издательство,Издательский бренд,...,Количество страниц,Размер,Тип обложки,Тираж,"Вес, г",Возрастные ограничения,Переводчик,Художник,Тип комикса,Разделы комикса
0,https://www.chitai-gorod.ru/product/skorb-sata...,312 ₽,400 ₽,4.1,925 оценок,Скорбь Сатаны,Мария Корелли,2772938,АСТ,Neoclassic,...,512,2.1x11.5x18,Мягкий переплёт,30000,250,16+,NaN,NaN,NaN,NaN
1,https://www.chitai-gorod.ru/product/1984-novyy...,312 ₽,400 ₽,4.1,1341 оценка,1984 (новый перевод),Джордж Оруэлл,2918634,АСТ,Neoclassic,...,320,2.4x11.5x18,Мягкий переплёт,30000,200,16+,NaN,NaN,NaN,NaN
2,https://www.chitai-gorod.ru/product/prestuplen...,303 ₽,388 ₽,4.2,1288 оценок,Преступление и наказание,Федор Достоевский,2465295,АСТ,Neoclassic,...,672,2.5x17.5x18,Мягкий переплёт,30000,319,12+,NaN,NaN,NaN,NaN
3,https://www.chitai-gorod.ru/product/chelovek-n...,284 ₽,365 ₽,4.5,363 оценки,Человек недостойный,Дадзай Осаму,3032403,АСТ,Neoclassic,...,192,1.5x11.5x18,Мягкий переплёт,25000,132,16+,NaN,NaN,NaN,NaN
4,https://www.chitai-gorod.ru/product/tri-tovari...,413 ₽,530 ₽,4.2,1269 оценок,Три товарища,Эрих Ремарк,2666861,АСТ,Neoclassic,...,480,2.5x11.6x18.1,Мягкий переплёт,30000,300,12+,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,https://www.chitai-gorod.ru/product/drakula-sa...,2207 ₽,2831 ₽,3.7,53 оценки,Дракула. Самая полная версия. Коллекционное ил...,Брэм Стокер,2976255,Эксмо,Издательство Алгоритм,...,448,2.6x17x24,Твёрдый переплёт,1500,760,16+,NaN,NaN,NaN,NaN
3596,https://www.chitai-gorod.ru/product/vremya-i-b...,1103 ₽,1415 ₽,4.3,42 оценки,Время и боги. Дочь короля Эльфландии,Лорд Дансейни,2958378,Иностранка,NaN,...,784,3.8x14.5x21.7,Твёрдый переплёт,3000,860,16+,NaN,NaN,NaN,NaN
3597,https://www.chitai-gorod.ru/product/roza-mira-...,1011 ₽,1297 ₽,3.8,29 оценок,Роза Мира,Даниил Андреев,2903592,Эксмо,NaN,...,704,3.4x14x20.6,Твёрдый переплёт,2000,638,16+,NaN,NaN,NaN,NaN
3598,https://www.chitai-gorod.ru/product/priyut-gre...,505 ₽,648 ₽,4.0,27 оценок,Приют Грез (новый перевод),Эрих Ремарк,2946441,АСТ,Neoclassic,...,256,1.8x13x20.5,Твёрдый переплёт,1500,230,16+,NaN,NaN,NaN,NaN


In [19]:
save_data(result, OUTPUT_FILE=OUTPUT_FILE)